In [44]:
import ccxt 
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from dotenv import load_dotenv

load_dotenv()

MONGO_URI = os.getenv("MONGO_URI")
FTX_API_KEY = os.getenv("FTX_API_KEY")
FTX_API_SECRET = os.getenv("FTX_API_SECRET")
SUBACCOUNT = 'test'

ftx = ccxt.ftx(
    {
        'apiKey': FTX_API_KEY,
        'secret': FTX_API_SECRET,
        'enableRateLimit': True,
        'headers': {
            'FTX-SUBACCOUNT': SUBACCOUNT,
        },
    }
)


In [45]:
data = ftx.fetch_my_trades(limit=10000)
trades = pd.DataFrame(data)
display(trades)

,info,timestamp,datetime,symbol,id,order,type,takerOrMaker,side,price,amount,cost,fee,fees
0,"{'id': '10536964044', 'market': 'DOGE/USD', 'f...",1666730783986,2022-10-25T20:46:23.986Z,DOGE/USD,10536964044,193469510259,None,maker,buy,0.062725,100.0,6.272500,"{'cost': -0.0005, 'currency': 'DOGE', 'rate': ...","[{'currency': 'DOGE', 'cost': -0.0005, 'rate':..."
1,"{'id': '10536967464', 'market': 'DOGE/USD', 'f...",1666730791602,2022-10-25T20:46:31.602Z,DOGE/USD,10536967464,193469539425,None,maker,buy,0.062715,100.0,6.271500,"{'cost': -0.0005, 'currency': 'DOGE', 'rate': ...","[{'currency': 'DOGE', 'cost': -0.0005, 'rate':..."
2,"{'id': '10536967688', 'market': 'DOGE/USD', 'f...",1666730792246,2022-10-25T20:46:32.246Z,DOGE/USD,10536967688,193469570648,None,maker,buy,0.062720,100.0,6.272000,"{'cost': -0.0005, 'currency': 'DOGE', 'rate': ...","[{'currency': 'DOGE', 'cost': -0.0005, 'rate':..."
3,"{'id': '10536969560', 'market': 'DOGE/USD', 'f...",1666730799660,2022-10-25T20:46:39.660Z,DOGE/USD,10536969560,193469596592,None,maker,buy,0.062670,100.0,6.267000,"{'cost': -0.0005, 'currency': 'DOGE', 'rate': ...","[{'currency': 'DOGE', 'cost': -0.0005, 'rate':..."
4,"{'id': '10536974671', 'market': 'DOGE-PERP', '...",1666730814726,2022-10-25T20:46:54.726Z,DOGE/USD:USD,10536974671,193469620936,None,maker,sell,0.062690,100.0,6.269000,"{'cost': -3.1345e-05, 'currency': 'USD', 'rate...","[{'currency': 'USD', 'cost': -3.1345e-05, 'rat..."
5,"{'id': '10536975758', 'market': 'DOGE-PERP', '...",1666730815251,2022-10-25T20:46:55.251Z,DOGE/USD:USD,10536975758,193469602265,None,maker,sell,0.062695,100.0,6.269500,"{'cost': -3.13475e-05, 'currency': 'USD', 'rat...","[{'currency': 'USD', 'cost': -3.13475e-05, 'ra..."
6,"{'id': '10536977673', 'market': 'DOGE-PERP', '...",1666730824620,2022-10-25T20:47:04.620Z,DOGE/USD:USD,10536977673,193469734737,None,maker,sell,0.062700,100.0,6.270000,"{'cost': -3.135e-05, 'currency': 'USD', 'rate'...","[{'currency': 'USD', 'cost': -3.135e-05, 'rate..."
7,"{'id': '10536978015', 'market': 'DOGE-PERP', '...",1666730824903,2022-10-25T20:47:04.903Z,DOGE/USD:USD,10536978015,193469588550,None,maker,sell,0.062710,100.0,6.271000,"{'cost': -3.1355e-05, 'currency': 'USD', 'rate...","[{'currency': 'USD', 'cost': -3.1355e-05, 'rat..."
8,"{'id': '10536982731', 'market': 'DOGE/USD', 'f...",1666730841463,2022-10-25T20:47:21.463Z,DOGE/USD,10536982731,193469822261,None,maker,buy,0.062665,100.0,6.266500,"{'cost': -0.0005, 'currency': 'DOGE', 'rate': ...","[{'currency': 'DOGE', 'cost': -0.0005, 'rate':..."
9,"{'id': '10536982919', 'market': 'DOGE-PERP', '...",1666730841504,2022-10-25T20:47:21.504Z,DOGE/USD:USD,10536982919,193469814828,None,maker,buy,0.062665,100.0,6.266500,"{'cost': -3.13325e-05, 'currency': 'USD', 'rat...","[{'currency': 'USD', 'cost': -3.13325e-05, 'ra..."


In [46]:
trades_cleaned = pd.json_normalize(trades['info'])
trades = pd.concat([trades.drop(['info'], axis=1), trades_cleaned], axis=1)
trades['price'] = trades['price'].astype(float)
trades['size'] = trades['size'].astype(float)
trades_cleaned = trades_cleaned.drop(['id', 'fee', 'feeCurrency'], axis=1)
display(trades_cleaned)



,market,future,baseCurrency,quoteCurrency,type,side,price,size,orderId,time,tradeId,feeRate,liquidity
0,DOGE/USD,None,DOGE,USD,order,buy,0.062725,100.0,193469510259,2022-10-25T20:46:23.986863+00:00,5219078235,-5e-6,maker
1,DOGE/USD,None,DOGE,USD,order,buy,0.062715,100.0,193469539425,2022-10-25T20:46:31.602954+00:00,5219079914,-5e-6,maker
2,DOGE/USD,None,DOGE,USD,order,buy,0.06272,100.0,193469570648,2022-10-25T20:46:32.246623+00:00,5219080026,-5e-6,maker
3,DOGE/USD,None,DOGE,USD,order,buy,0.06267,100.0,193469596592,2022-10-25T20:46:39.660818+00:00,5219080929,-5e-6,maker
4,DOGE-PERP,DOGE-PERP,None,None,order,sell,0.06269,100.0,193469620936,2022-10-25T20:46:54.726528+00:00,5219083401,-5e-6,maker
5,DOGE-PERP,DOGE-PERP,None,None,order,sell,0.062695,100.0,193469602265,2022-10-25T20:46:55.251379+00:00,5219083940,-5e-6,maker
6,DOGE-PERP,DOGE-PERP,None,None,order,sell,0.0627,100.0,193469734737,2022-10-25T20:47:04.620576+00:00,5219084875,-5e-6,maker
7,DOGE-PERP,DOGE-PERP,None,None,order,sell,0.06271,100.0,193469588550,2022-10-25T20:47:04.903177+00:00,5219085046,-5e-6,maker
8,DOGE/USD,None,DOGE,USD,order,buy,0.062665,100.0,193469822261,2022-10-25T20:47:21.463537+00:00,5219087330,-5e-6,maker
9,DOGE-PERP,DOGE-PERP,None,None,order,buy,0.062665,100.0,193469814828,2022-10-25T20:47:21.504368+00:00,5219087424,-5e-6,maker


In [47]:
df = trades_cleaned

start = pd.to_datetime(df['time'].min())
end = pd.to_datetime(df['time'].max())
delta = end - start

spot = df.loc[df['future'].isnull()]
perp = df.loc[df['future'].notnull()]

spot['price'] = spot['price'].astype(float)
spot['size'] = spot['size'].astype(float)
perp['price'] = perp['price'].astype(float)
perp['size'] = perp['size'].astype(float)
perp['side']

perp.loc[perp['side'] == 'buy', 'size'] = -perp['size']
spot.loc[spot['side'] == 'sell', 'size'] = -spot['size']

print(spot['size'])

# get the start and end time delta in the df 
start = pd.to_datetime(df['time'].min())
end = pd.to_datetime(df['time'].max())
delta = end - start

spot_avg_entry = np.average(spot['price'], weights=spot['size'])
perp_avg_entry = np.average(perp['price'], weights=perp['size'])
entry_spread = round((perp_avg_entry - spot_avg_entry)*100, 6)
spread_entry_gain_loss_percent = round((entry_spread / perp_avg_entry), 6) 

data = {
    'spot_avg_entry': spot_avg_entry, 
    'perp_avg_entry': perp_avg_entry,
    'entry_spread': entry_spread,
    'spread_entry_gain_loss_percent': spread_entry_gain_loss_percent,
    'spot_market': spot['market'].unique()[0],
    'perp_market': perp['market'].unique()[0],
    'spot_size': spot['size'].sum(),
    'perp_size': perp['size'].sum(),
    'start': start,
    'end': end,
    'entry_total_time': delta,
}

0     100.0
1     100.0
2     100.0
3     100.0
8     100.0
15    100.0
16    100.0
17   -100.0
18     -4.0
20    -96.0
Name: size, dtype: float64


In [48]:
# print(data)

# {'spot_avg_entry': 0.06269000000000001, 'perp_avg_entry': 0.06267611111111111, 'entry_spread': -0.001389, 'spread_entry_gain_loss_percent': -0.022162, 'spot_market': 'DOGE/USD', 'perp_market': 'DOGE-PERP', 'spot_size': 700.0, 'perp_size': 900.0, 'start': Timestamp('2022-10-25 20:46:23.986863+0000', tz='UTC'), 'end': Timestamp('2022-10-25 20:47:47.687975+0000', tz='UTC'), 'entry_total_time': Timedelta('0 days 00:01:23.701112')}

In [49]:
print(data)

{'spot_avg_entry': 0.062719, 'perp_avg_entry': 0.06269301000000001, 'entry_spread': -0.002599, 'spread_entry_gain_loss_percent': -0.041456, 'spot_market': 'DOGE/USD', 'perp_market': 'DOGE-PERP', 'spot_size': 500.0, 'perp_size': 500.0, 'start': Timestamp('2022-10-25 20:46:23.986863+0000', tz='UTC'), 'end': Timestamp('2022-10-25 20:52:08.659108+0000', tz='UTC'), 'entry_total_time': Timedelta('0 days 00:05:44.672245')}
